In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.linear_model import LogisticRegression

from scipy.optimize import minimize

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [ ]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [ ]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Mounted at /content/drive


In [ ]:
algorithm_name = 'lrcv'
feature_name = 'stacking3'
model_name = f'{algorithm_name}_{feature_name}_1'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [ ]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [ ]:
def load_data(model_names, oof_list, test_list, feature_names=None):
    for model in model_names:
        oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
        test_list.append(np.loadtxt(test_dir / f'{model}_test_pred.csv',delimiter=','))
        if feature_names != None:
            feature_names += {f'{model}_class0',f'{model}_class1',f'{model}_class2'}

In [ ]:
all_oof = []
all_test = []
feature_names = []

model_names = ['lrcv_stacking2_1','xgbcv_stacking2_1']
load_data(model_names,all_oof, all_test, feature_names)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 6),
 (80000, 6),
 ['lrcv_stacking2_1_class0',
  'lrcv_stacking2_1_class1',
  'lrcv_stacking2_1_class2',
  'xgbcv_stacking2_1_class0',
  'xgbcv_stacking2_1_class1',
  'xgbcv_stacking2_1_class2'])

In [ ]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [ ]:
# Logistic Regression parameters
lr_params = {
    'multi_class': 'multinomial',
    'penalty': 'elasticnet',
    'l1_ratio': 0.01,
    'solver': 'saga',
    'C': 1.0,
    'max_iter': 100000,
    'class_weight': None,
    'verbose': 0,
    'n_jobs': -1,
    'random_state': SEED
}

In [ ]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    print(f'Traning model for CV #{fold+1}')
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    lr_clf = LogisticRegression(**lr_params)
    lr_clf.fit(X_train,y_train)

    stk_test_pred += lr_clf.predict_proba(all_test) / n_splits
    stk_oof_pred[val_idx] += lr_clf.predict_proba(X_val)

    mlogloss.append(log_loss(y_val,stk_oof_pred[val_idx]))
    print(f'{fold+1} logloss = {log_loss(y_val,stk_oof_pred[val_idx])}')

print('mean logloss= ', np.mean(mlogloss))

Traning model for CV #1
1 logloss = 0.16666616140233328
Traning model for CV #2
2 logloss = 0.16393185344925376
Traning model for CV #3
3 logloss = 0.16332488646593474
Traning model for CV #4
4 logloss = 0.16743316563196498
Traning model for CV #5
5 logloss = 0.16704226347323492
mean logloss=  0.16567966608454432


# Ensemble

- cv score가 안정적이여서 Stacking 예측값와 함께 가중치 최적화함
- 가중치 최적화에는 scipy의 minimize 사용

In [ ]:
def log_loss_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, oof_predictions):
        final_prediction += weight*prediction

    return log_loss(y, final_prediction)

In [ ]:
oof_predictions = []
test_predictions = []

model_names = ['lrcv_stacking2_1','xgbcv_stacking2_1']
load_data(model_names,oof_predictions, test_predictions)

oof_predictions.append(stk_oof_pred)
test_predictions.append(stk_test_pred)
len(oof_predictions), oof_predictions, len(test_predictions), test_predictions

(3, [array([[9.99363449e-01, 1.37222895e-04, 4.99327696e-04],
         [3.71178576e-03, 5.16462639e-01, 4.79825575e-01],
         [9.99784727e-01, 8.63032014e-05, 1.28969328e-04],
         ...,
         [9.99363448e-01, 1.37223364e-04, 4.99328406e-04],
         [9.99849705e-01, 4.53759554e-05, 1.04918762e-04],
         [9.99363673e-01, 1.37181133e-04, 4.99146174e-04]]),
  array([[9.99993801e-01, 4.41699103e-06, 1.76609967e-06],
         [2.60089734e-03, 4.32234049e-01, 5.65165043e-01],
         [9.99967575e-01, 2.38924331e-05, 8.54993232e-06],
         ...,
         [9.99993801e-01, 4.41699103e-06, 1.76609967e-06],
         [9.99995828e-01, 1.86849604e-06, 2.24750897e-06],
         [9.99993801e-01, 4.41699103e-06, 1.76609967e-06]]),
  array([[9.99250875e-01, 3.42255235e-04, 4.06869453e-04],
         [2.25569889e-03, 4.14133510e-01, 5.83610791e-01],
         [9.99291505e-01, 3.68394352e-04, 3.40100415e-04],
         ...,
         [9.99250875e-01, 3.42255237e-04, 4.06869456e-04],
       

In [ ]:
best_scores = []
weights = []

iteration = 100 # 여러 번 구한 가중치 값 중 Best score를 선택
for i in tqdm(range(iteration)):
    starting_values = np.random.uniform(size=len(oof_predictions))
    bounds = [(0,1)]*len(oof_predictions)
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})

    res = minimize(log_loss_func,
                   starting_values,
                   method = 'SLSQP',
                   bounds = bounds,
                   constraints = cons,
                   options={'maxiter': 500})
    best_scores.append(res['fun'])
    weights.append(res['x'])

weights = weights[np.argmin(best_scores)]
print('\n')
print('Ensemble score: {}'.format(np.min(best_scores)))
print('Best Weights: {}'.format(weights))

100%|██████████| 100/100 [06:22<00:00,  3.82s/it]



Ensemble score: 0.1522391851876869
Best Weights: [4.29583514e-01 5.70416486e-01 1.90711162e-16]


In [ ]:
final_prediction = 0

# 가중치 적용
for weight, p in zip(weights, test_predictions):
    final_prediction += weight*p

print(final_prediction)

[[6.11706601e-05 1.17390696e-01 8.82548136e-01]
 [9.99893901e-01 3.68334256e-05 6.92581134e-05]
 [4.82721591e-05 8.11027361e-02 9.18848987e-01]
 ...
 [9.99906904e-01 3.01832218e-05 6.29051788e-05]
 [7.81495408e-05 5.23609762e-02 9.47560884e-01]
 [9.99906721e-01 3.03233417e-05 6.29452110e-05]]


# 제출 파일 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(final_prediction, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [ ]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [ ]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')